In [70]:
# Packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [126]:
# Make a map of league number code to league name
url = f'https://fbref.com/en/comps/0/history/'
page = requests.get(url)
#header = page.text.split('</h1>')[0].split('<h1>')[-1]
#' '.join(header.split('Seasons')[0].split('\t')[-1].split())
soup = BeautifulSoup(page.content, 'lxml')
while soup.find('tr', class_='thead') is not None:
    soup.find('tr', class_='thead').decompose()
table = soup.find('table', id = 'comps_1_fa_club_league_senior')
comps = pd.read_html(str(table))[0]
comps['CODE'] = [int(x['href'].split('/')[3]) for x in list(table.find_all('a', href = True)) if 'comps' in x['href']][0::3]
comps = comps.groupby('Gender').get_group('M')
comps['Country'] = comps['Country'].apply(lambda x: x.split()[-1])
comps['acyear'] = comps['First Season'].apply(lambda x: len(x) > 4)
comps['First Season'] = comps['First Season'].apply(lambda x: x[-4:]).astype(int)
comps = comps.rename(columns = {'Competition Name': 'Competition'}).reset_index(drop = True)
comps['title'] = comps['Country'] + ' ' + comps['Competition'].apply(lambda x: x.lower())
comps = comps[['Competition', 'title', 'Country', 'First Season', 'acyear', 'CODE']]
comps.to_csv('competitions.csv', index = None)

In [30]:
from scraping import scraper

In [31]:
Scraper = scraper([2021], 23)
df = Scraper.fit()

KeyError: False

In [12]:
df.Date.max().strftime('%d %b')

'29 Sep'

In [27]:
a['CODE'][0]

9

In [28]:
import time

for i in range(10):
    print(f"Counting: {i}", end="\r")
    time.sleep(1)


In [43]:
# Scrape looped year's schedule
url = f'https://fbref.com/en/comps/23/2020-2021/schedule/'
soup = BeautifulSoup(requests.get(url).content, 'lxml') # Create a soup object from the webpage
while soup.find('tr', class_ = 'thead') is not None: # Decompose all headers
    soup.find('tr', class_ = 'thead').decompose()
if soup.find('table') is not None :
        
    data = pd.read_html(str(soup.find('table')))[0] # Read table in a dataframe
    data = data[data['Home'].notna()] # Remove headers
    if 'Notes' in data.columns:
        data = data[data['Notes'].isna()]
    data = data[['Date', 'Home', 'Away', 'Score']] # Only keep necessary columns
    data['Date'] = pd.to_datetime(data['Date']) # Turn dates into datetime format
    data['G_home'] = data['Score'].apply(lambda x: int(str(x).split('–')[0])  if (np.all(pd.notnull(x))) else x) # Set home goals
    data['G_away'] = data['Score'].apply(lambda x: int(str(x).split('–')[-1]) if (np.all(pd.notnull(x))) else x) # Set away goals

In [48]:
# Create list of teams competing this season
teams = sorted(list(set(data['Home'])))
homes, aways = [], [] # Initiate empty lists of home & away games
# For each team keep their games and compute the rolling -- with varying windows -- statistics
# Separate into home and away games
for tm in teams:
    print(tm)

    df = data.copy()[(data['Home'] == tm) | (data['Away'] == tm)].sort_values('Date').reset_index(drop = True) # Team results
    df['H'] = df['Home'] == tm # Differentiate home & away games
    df['A'] = df['H']    == False
    df['Team'] = len(df) * [tm] # Specify home team
    df['Opp'] = df['Home'] * df['A'] + df['Away'] * df['H'] # Specify name of opponent
    df['GF'] = df['G_home'] * df['H'] + df['G_away'] * df['A'] # Goals for
    df['GA'] = df['G_home'] * df['A'] + df['G_away'] * df['H'] # Goals against
    df['GD'] = df['GF'] - df['GA'] # Goal difference
    df['W'] = np.sign(df['GD']) # Win (1), Draw (0) or Loss (-1)
    df = df[['Date', 'Team', 'Opp', 'H', 'W', 'GF', 'GA', 'GD']] # Keep relevant columns
    df['PTS%'] = [np.nan] + list(np.cumsum(df['W'].apply(lambda x: {-1:0,0:1,1:3}.get(x))) / (3*(df.index + 1)))[:-1] # Calculate the % of max. PTS
    features = ['PTS%'] # Add it as a features
    for f in ['W', 'GF', 'GA', 'GD']: # Loop for all features
        for w in [1,2,5,10]: # and windows
            # Compute the lagged feature
            df[f'{f}_{w}'] = [np.nan] + list(df[f].rolling(w, min_periods = 1).mean())[:-1]
            features.append(f'{f}_{w}') # Add lagged features to features list
        # Compute the rolling mean for each feature
        df[f'{f}_inf'] = [np.nan] + list(df[f].rolling(1000, min_periods = 1).mean())[:-1]
        features.append(f'{f}_inf') # Add it again to the list of features
    df['Rest'] = [np.nan] + list(np.diff(df['Date']).astype(float) / (10**9) / (60 * 60 * 24)) # Compute the number of days of rest
    features.append('Rest') # Add to list of features
    df['Game'] = df.index + 1 # Specify the number of games played

    # Cluster home games and rename columns accordingly
    home = df.groupby('H').get_group(True)
    renameDict = {'Team': 'Home', 'Opp': 'Away'}
    for f in ['W', 'GF', 'GA', 'GD', 'Game'] + features:
        renameDict[f] = f'{f}_home'
    home = home.rename(columns = renameDict)
    # Cluster away games and rename columns accordingly
    away = df.groupby('H').get_group(False)
    renameDict = {'Team': 'Away', 'Opp': 'Home'}
    for f in ['W', 'GF', 'GA', 'GD', 'Game'] + features:
        renameDict[f] = f'{f}_away'
    away = away.rename(columns = renameDict)
    away = away.drop(columns = ['H', 'W_away', 'GF_away', 'GA_away', 'GD_away']) # Drop redundant columns
    # Append home & away games to appropriate lists
    homes.append(home)
    aways.append(away)

ADO Den Haag
AZ Alkmaar
Ajax
Almere City


KeyError: False

In [49]:
df

,Date,Team,Opp,H,W,GF,GA,GD,PTS%,W_1,...,GA_5,GA_10,GA_inf,GD_1,GD_2,GD_5,GD_10,GD_inf,Rest,Game
0,2021-05-15,Almere City,NEC Nijmegen,True,-1,0,4,-4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [14]:
from bs4 import BeautifulSoup

In [17]:
import requests
import pandas as pd

In [19]:
data

,Date,Home,Away,Score
0,2022-07-22,Standard Liège,Gent,2–2
1,2022-07-23,Charleroi,Eupen,3–1
2,2022-07-23,Kortrijk,OH Leuven,0–2
3,2022-07-23,Zulte Waregem,R.F.C. Seraing,2–0
4,2022-07-23,Sint-Truiden,Union SG,1–1
...,...,...,...,...
365,2023-05-28,Club Brugge,Genk,NaN
367,2023-06-03,Gent,Standard Liège,NaN
368,2023-06-03,Cercle Brugge,Westerlo,NaN
369,2023-06-04,Union SG,Club Brugge,NaN


In [52]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from search_engine import search
from scraping import scraper
from datetime import datetime, timedelta
from train import training, loading
import numpy as np
import pandas as pd

print('\n')
# PART 1 :   Ask the user for a competition
query = input('Enter competition :   \n\t')
comp, flag = search(query)
print('\n')


if flag :

    st, code, name, country = comp['First Season'][0], comp['CODE'][0], comp['Competition'][0], comp['Country'][0]
    # PART 2 :   Scrape games to be played in the next week
    season = 2023
    print('Scraping next few games ...')
    Scraper = scraper(years = [season], code = code)
    games = Scraper.fit(verbose = False)
    # Keep games yet to be played between now and a week later
    now, then = datetime.today().strftime('%Y-%m-%d'), datetime.now() + timedelta(days = 7)
    games = games[(games['Date'] >= now) & (games['Date'] <= then)]
    games = games[(games['Result'].isna()) & (games['PTS%'].notna())]
    games = games[['Date', 'Home', 'Away'] + Scraper.features].reset_index(drop = True)

    # Stop if there are no games scheduled in the upcoming week
    if len(games) == 0 :
        print(f'No {name} ({country}) games scheduled in the next week ...')


    # PART 3 :   Predict the results of the upcoming games
    else :

        try :     # Try loading the model of the competition
            model = loading(code)

        except :  # If the model doesn't yet exist, train one and save it
            model = training(code, st)

        # Predict the result of the upcoming games and append the results to the dataframe
        X_pred = games[Scraper.features]
        preds = model.predict(X_pred, verbose = 0)
        games['PRED.'] = list(pd.Series(np.argmax(preds, axis = 1)).replace(0, 'HOME').replace(1, 'DRAW').replace(2, 'AWAY'))
        games[['HOME_W', 'DRAW', 'AWAY_W']] = np.round(preds / preds.sum(axis = 1)[:, None], 2)
        games = games[['Date', 'Home', 'Away', 'PRED.', 'HOME_W', 'DRAW', 'AWAY_W']].rename(columns = {'Date':'DATE', 'Home':'HOME', 'Away':'AWAY'}).set_index('DATE')

        # Print the results
        print('\n')
        print(f'{name.upper()} ({country.upper()}) PREDICTIONS ({games.index.min().strftime("%d %b")} - {games.index.max().strftime("%d %b")})\n')
        print(games)
        print('\n\n')





Scraping next few games ...
Loading neural network model ...


ARGENTINE PRIMERA DIVISIÓN (ARG) PREDICTIONS (30 Apr - 07 May)

                        HOME                                AWAY PRED.  \
DATE                                                                     
2023-04-30          Talleres                        Rosario Cent  HOME   
2023-04-30    Defensa y Just                               Colón  HOME   
2023-04-30          CA Unión                               Lanús  HOME   
2023-04-30     Independiente              Club Atlético Belgrano  HOME   
2023-05-07             Tigre                         Newell's OB  HOME   
2023-05-07        CC Córdoba                           Sarmiento  HOME   
2023-05-07       Estudiantes                         Vélez Sarsf  HOME   
2023-05-07           Arsenal                        Gimnasia ELP  HOME   
2023-05-07       River Plate                        Boca Juniors  HOME   
2023-05-07  Barracas Central  Instituto Atlético Centra

In [119]:
train = pd.read_csv('training_data/TRAINING_DATA_COMP21.csv')

In [120]:
# Clean training data -- at least 5 games played in the season + no games from COVID-19 1st lockdown
train = train[(train['Game_home'] > 4) & (train['Game_away'] > 4) & (train['Rest'].abs() <= 40)]

In [69]:
train.sort_values('Date')

,Unnamed: 0,Date,Season,Game_home,Game_away,Home,Away,Result,G_home,G_away,...,GA_inf,GD_1,GD_2,GD_5,GD_10,GD_inf,Rest,Draw,Loss,Win
60,60,2016-09-30,2017,5,5,Quilmes,Club Olimpo,1.0,2.0,1.0,...,1.500000,-6.0,-3.0,-1.5,-1.5,-1.500000,-2.0,0,0,1
61,61,2016-09-30,2017,5,5,Godoy Cruz,CA Unión,1.0,2.0,1.0,...,1.000000,-1.0,-1.0,-1.0,-1.0,-1.000000,2.0,0,0,1
62,62,2016-10-01,2017,5,5,River Plate,Vélez Sarsf,1.0,3.0,0.0,...,-0.500000,3.0,2.0,2.0,2.0,2.000000,-1.0,0,0,1
63,63,2016-10-01,2017,5,5,Rosario Cent,Arsenal,1.0,5.0,0.0,...,-1.000000,0.0,0.5,0.5,0.5,0.500000,2.0,0,0,1
64,64,2016-10-01,2017,5,5,Aldosivi,Talleres,1.0,2.0,1.0,...,0.500000,0.0,-0.5,-0.5,-0.5,-0.500000,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2907,2907,2023-07-30,2015,27,27,River Plate,Racing Club,NaN,NaN,NaN,...,-0.928571,NaN,NaN,NaN,NaN,1.500000,0.0,0,0,0
2908,2908,2023-07-30,2016,27,27,Arsenal,Colón,NaN,NaN,NaN,...,0.423077,NaN,NaN,NaN,NaN,-0.489011,0.0,0,0,0
2909,2909,2023-07-30,2016,27,27,CA Huracán,Vélez Sarsf,NaN,NaN,NaN,...,0.428571,NaN,NaN,NaN,NaN,-0.500000,0.0,0,0,0
2911,2911,2023-07-30,2016,27,27,Newell's OB,Talleres,NaN,NaN,NaN,...,0.010989,NaN,NaN,NaN,NaN,-0.835165,0.0,0,0,0


In [80]:
'hi' * False

''

In [92]:
year = 2022
acyear = True

In [93]:
f'{year-1}-' * acyear + str(year)

'2021-2022'

In [123]:
idx = np.array([f in features for f in train.columns])

In [124]:
train = np.array(train)

In [125]:
train[:,idx]

array([[-0.25, -2.0, -1.0, ..., -1.5, -1.5, -2.0],
       [-0.25, 0.0, -0.5, ..., -1.0, -1.0, 2.0],
       [0.4166666666666666, 1.0, 1.0, ..., 2.0, 2.0, -1.0],
       ...,
       [nan, nan, nan, ..., nan, -0.5, 0.0],
       [nan, nan, nan, ..., nan, -0.489010989010989, 0.0],
       [nan, nan, nan, ..., nan, 0.2142857142857142, 0.0]], dtype=object)

In [105]:
features

['PTS%',
 'W_1',
 'W_2',
 'W_5',
 'W_10',
 'W_inf',
 'GF_1',
 'GF_2',
 'GF_5',
 'GF_10',
 'GF_inf',
 'GA_1',
 'GA_2',
 'GA_5',
 'GA_10',
 'GA_inf',
 'GD_1',
 'GD_2',
 'GD_5',
 'GD_10',
 'GD_inf',
 'Rest']

In [107]:
train.columns

Index(['Unnamed: 0', 'Date', 'Season', 'Game_home', 'Game_away', 'Home',
       'Away', 'Result', 'G_home', 'G_away', 'PTS%', 'W_1', 'W_2', 'W_5',
       'W_10', 'W_inf', 'GF_1', 'GF_2', 'GF_5', 'GF_10', 'GF_inf', 'GA_1',
       'GA_2', 'GA_5', 'GA_10', 'GA_inf', 'GD_1', 'GD_2', 'GD_5', 'GD_10',
       'GD_inf', 'Rest', 'Draw', 'Loss', 'Win'],
      dtype='object')